In [1]:
from sklearn.datasets import load_wine
import pandas as pd

wine = load_wine()
pd.DataFrame(wine.data, columns=wine.feature_names)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


In [3]:
#독립변수
X = wine.data
#종속변수
y = wine.target

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [4]:
import torch
# 학습용 데이터를 텐서로 변환
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).long()

# 검증용 데이터를 텐서로 변환
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()

In [5]:
# 독립변수와 종속변수의 텐서를 합침
from torch.utils.data import DataLoader, TensorDataset
train = TensorDataset(X_train, y_train)
print(train[0])

# 미니배치로 분할
train_loader = DataLoader(train, batch_size=16, shuffle=True)

(tensor([1.2700e+01, 3.8700e+00, 2.4000e+00, 2.3000e+01, 1.0100e+02, 2.8300e+00,
        2.5500e+00, 4.3000e-01, 1.9500e+00, 2.5700e+00, 1.1900e+00, 3.1300e+00,
        4.6300e+02]), tensor(1))


In [6]:
# 신경망 구성
from torch import nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 96) # input 13, output 96
        self.fc2 = nn.Linear(96, 96)
        self.fc3 = nn.Linear(96, 96)
        self.fc4 = nn.Linear(96, 96)
        self.fc5 = nn.Linear(96, 96)
        self.fc6 = nn.Linear(96, 3)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x)

# 인스턴스 생성
model = Net() #.cuda()

In [8]:
import torch.optim as optim
from torch.autograd import Variable

# 손실함수 객체
criterion = nn.CrossEntropyLoss()

# 최적화함수
optimizer = optim.SGD(model.parameters(), lr=0.001)

# 학습 시작
for epoch in range(500):
    total_loss = 0
    for train_x, train_y in train_loader:
        # 계산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        #텐서를 gpu로 이동시킴
        train_x=train_x
        train_y=train_y
        # 경사 초기화
        optimizer.zero_grad()
        # 순전파 계산
        output = model(train_x)
        # 오차계산
        loss = criterion(output, train_y)
        # 역전파 계산
        loss.backward()
        # 가중치 업데이트
        optimizer.step()
        # 누적 오차 계산
        total_loss += loss.data
    # 50회 반복마다 누적오차 출력
    if (epoch+1) % 50 == 0:
        print(epoch+1, total_loss)

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_2020\3710233476.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


50 tensor(6.7881)
100 tensor(6.9884)
150 tensor(7.0122)
200 tensor(6.4394)
250 tensor(6.0861)
300 tensor(6.2576)
350 tensor(6.0031)
400 tensor(6.4068)
450 tensor(6.3385)
500 tensor(5.9687)


In [9]:
# 계산 그래프 구성
X_test, y_test = Variable(X_test), Variable(y_test)

# 출력값 계산
result = torch.max(model(X_test).data, 1)[1]

# gpu에 저장된 텐서를 cpu로 이동시킴
# y_test=y_test.cpu()
# result=result.cpu()

# 모형의 정확도 측정
accuracy = sum(y_test.data.numpy() == result.numpy()) / len(y_test.data.numpy())

# 모형의 정확도 출력
accuracy

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_2020\3710233476.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


0.7222222222222222